# Introdução Machine Learning - Data ICMC-USP

## Prática - k-Nearest Neighbors

Esse material foi desenvolvido pelo **Data**, grupo de extensão de aprendizado e ciência de dados compostos por alunos do Instituto de Ciências Matemáticas e de Computação da USP

Para saber mais sobre as atividades do Data entre no nosso site e nos siga e nossas redes sociais:
- [Site](http://data.icmc.usp.br/)
- [Twitter](https://twitter.com/data_icmc)
- [LinkedIn](https://www.linkedin.com/school/data-icmc/)
- [Facebook](https://www.facebook.com/dataICMC/)

Aproveite o material!

In [1]:
import numpy as np
import pandas as pd

Vamos começar carregando os dados que iremos usar no nossa tarefa. Esses dados fornecem várias informações a respeito de diferentes vinhos e o objetivo é classificar se o vinho é bom (target é a coluna *is_good*).

Esse conjunto de dados é uma modificação do conjunto 

In [2]:
df = pd.read_csv('vinho.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is good
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0


### Deixando os dados na mesma escala
Para vários algoritmos é importante deixarmos os dados em uma mesma escala, e o kNN um desses casos. Para entender melhor vamos olhar o exemplo a seguir:

<img src="grafico_escala.png" style="width: 400px"/>

Nesse caso a distância entre os dois pontos é dada por

$$
\begin{align*}
\text{dist}(x^{(1)}, x^{(2)}) &= \sqrt{(x^{(1)}_1 - x^{(2)}_1)^2 + (x^{(1)}_2 - x^{(2)}_2)^2} \\
  &= \sqrt{(3 - 2)^2 + (10000 - 9000)^2} \\
  &= \sqrt{1 + 1000000} \\
  &= \sqrt{1000001} \\
  &= 1000.0005
\end{align*}$$


Como as escalas são muito diferentes o primeiro atributo acaba não interferindo em praticamente nada no resultado da distância. E é importante perceber que esse tipo de situação ocorre com frequência em conjuntos de dados reais.

Existem diversas formas de tratar essa situação, aqui usaremos uma técnica chamada **Min-Max Scaling**, que transforma os dados deixando-os no intervalo $[0, 1]$. A formula é da transformação é a seguinte:

$$x^{(i)}_j \leftarrow \frac{x^{(i)}_j - min(x_j)}{max(x_j) - min(x_j)}$$

Em palavras significa que vamos subtrair o menor valor da atributo e dividir pela amplitude (diferença entre o máximo e o mínimo).


Pronto, agora que entendemos podemos fazer isso para todas as nossas colunas.

In [3]:
def MinMaxScale(x, min, max):
    return (x - min) / (max - min)

In [4]:
%%time
mins = df.min()
maxs = df.max()

for column in df: 
    df[column] = MinMaxScale(df[column], mins[column], maxs[column])

df.head()

CPU times: user 5.4 ms, sys: 1.23 ms, total: 6.62 ms
Wall time: 6.06 ms


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is good
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.0
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.0
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.0
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,1.0
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.0


In [5]:
target = "is good"
features = df.columns.to_list()
features.remove(target)

Como vimos na aula, é necessário separar o conjunto de dados em treino e validação:

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

Devemos implementar uma função de distância - neste exemplo, utilizaremos a distância euclidiana.

Dica: tente implementar uma nova função de distância, como por exemplo a distância Manhattan e verifique como varia a performance do nosso modelo KNN.

In [7]:
def distance(x, y):
    return np.sqrt(sum((x - y)**2))

Construindo a classe KNN e implementando seus métodos:

In [40]:
class KNearestNeighbors:
    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_val):
        # Lista para resultado
        y_pred = []
        # Para cada amostra de vale, vamos calcular sua distância até os dados de treino
        X_val_arr = X_val.to_numpy()
        X_train_arr = self.X_train.to_numpy()
        # print("Enumerate: ", enumerate(X_train_arr))
        for row_val in X_val_arr: #for each validation example
            # Array para guardar [index_train, distancia]
            distances = []
            # Vamos percorrer os dados de treino colocando as distancias em nossa matriz
            # print("Row val: ", row_val)
            for index_train, row_train in enumerate(X_train_arr):
                # print("Row val length: ", len(row_val), ", Index train: ", index_train, ", Row train length: ", len(row_train), ", Distance: ", distance(row_val, row_train))
                distances.append([index_train, distance(row_val, row_train)])
            # Vamos ordenar o array com base nas distancias (np.argsort retorna somente os indices)
            distances = sorted(distances, key=lambda x: x[1])
            # Pegando os k primeiros vizinhos
            nearestNeighbors = distances[0:self.k]
            # Ignorando agora as distancias
            nearestNeighbors = np.array(nearestNeighbors)[:,0]
            
            # Para cada vizinho, vamos analisar nos dados de treino o target
            result = []
            for neighbor in nearestNeighbors:
                result.append(y_train.to_numpy()[int(neighbor)])
            print(nearestNeighbors)
            print(result)
            if np.array(result).sum() > len(result) / 2:
                y_pred.append(1)
            else:
                y_pred.append(0)

        return y_pred

    def accuracy(self, y_val, y_pred):
        total = 0
        for val, pred in zip(y_val, y_pred):
            if (val == pred):
                total += 1
        return total / len(y_val)

Uma vez com a classe construída, podemos instanciar um modelo e testar sua acurácia.

In [41]:
model = KNearestNeighbors(k=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
model.accuracy(y_pred, y_val)

[750. 126. 962. 733. 794.]
[0.0, 1.0, 0.0, 0.0, 0.0]
[ 321.  315.  448. 1090.  414.]
[0.0, 0.0, 0.0, 0.0, 1.0]
[  34.  667. 1116.  285.  977.]
[1.0, 0.0, 0.0, 1.0, 0.0]
[1181.  155.  198.   54.  243.]
[0.0, 1.0, 1.0, 1.0, 1.0]
[ 157.  536. 1018. 1120.  830.]
[1.0, 1.0, 1.0, 0.0, 1.0]
[1239.  424. 1243.  323.  873.]
[0.0, 0.0, 1.0, 0.0, 0.0]
[1041. 1069. 1238.  240. 1008.]
[0.0, 0.0, 0.0, 0.0, 0.0]
[ 736.  168.  720.  935. 1186.]
[0.0, 0.0, 0.0, 0.0, 0.0]
[971. 339. 686. 981. 817.]
[1.0, 1.0, 1.0, 1.0, 1.0]
[1232. 1025.   15.  152.  568.]
[1.0, 1.0, 0.0, 0.0, 1.0]
[158.  87. 130. 407. 650.]
[1.0, 1.0, 1.0, 1.0, 1.0]
[184. 256. 641. 640. 835.]
[1.0, 0.0, 1.0, 1.0, 1.0]
[ 793.  372.  691. 1060.  268.]
[1.0, 1.0, 1.0, 0.0, 0.0]
[ 147. 1061.  164.  793.  372.]
[0.0, 0.0, 0.0, 1.0, 1.0]
[181. 307.  47. 392.  54.]
[1.0, 1.0, 0.0, 0.0, 1.0]
[ 743. 1075.  195.  100.  110.]
[1.0, 1.0, 1.0, 0.0, 1.0]
[1225.  861. 1113.  822.  991.]
[0.0, 1.0, 0.0, 0.0, 0.0]
[ 482.  617.  776.  914. 1089.]
[1.0, 1

0.684375

In [20]:
ks = [3, 5, 7, 9]
for k in ks:
  model = KNearestNeighbors(k=k)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  print(model.accuracy(y_pred, y_val))

0.71875
0.684375
0.709375
0.7
